# Designing  a data model

- don't put the same name for two string
-<img src = 'desgin_DB.png'>

# Representing Data Model

- start witht Track table. id 
- Primary key, each table must have an 'id' --> Primary key: one key for every row
- Foreign key, columns to add in tables tha have starting points, the end point is the 'primary key' of another table
- Logical key, unique thing to llok up from the outside world


<img src = 'represent_DB.png'>

### We work from the out to in
### WE USE THE SQLITEBROWSER
1.- Artist table

2.- Album table, has foreign key!
"""CREATE TABLE Album (
id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
artist_id INTEGER,
title TEXT
)"""

3.- Genre table
"""CREATE TABLE Genre (
id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
name TEXT
)"""

4.- Track table
CREATE TABLE Track (
id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
title TEXT,
artist_id INTEGER,
genre_id INTEGER,
len INTEGER, rating INTEGER, count INTEGER
)

# INSERT DATA
- go to sqllitebrowser

1.- Add data in Artist
"""insert into Artist (name) values ('Led Zeppelin')"""
"""insert into Artist (name) values ('AC/DC)"""

2.- Add data in Genre
insert into Genre (name) values ('Rock');
insert into Genre (name) values ('Metal')

3.- Add data in Artist (it has a primary key)
 this time we are doing it manually, so we have to remember the values '2' of artist_id is fo AC/ and '1' is for 'Led Zeppelin' --> there are better ways to do it!
insert into Album (title, artist_id) values ('Who Made Who', 2');
insert into Album (title, artist_id) values ('IV', 1');

4.- Add data to Track

insert into Track (title, rating, len, count, album_id, genre_id) values ('Black Dog', 5, 297, 0, 2,1);

insert into Track (title, rating, len, count, album_id, genre_id) values ('Stairway', 5, 481, 0, 2,1);

insert into Track (title, rating, len, count, album_id, genre_id) values ('About to Rock, 5, 313, 0, 1,2);

insert into Track (title, rating, len, count, album_id, genre_id) values ('Who Made Who', 5, 207, 0, 1, 2)
<img src = 'relations.png'>

# JOINs and RELATIONS

- JOIN operation
"""select Album.title, Artist.name from Album join Artist on Album.artist_id = Artist.id"""

- see the source of the connection, all tables
"""select Album.title, Album.artist_id, Artist.id, Artist.name from Album join Artist on Album.artist_id = Artist.id"""


- if  no 'on' clause --> all the combinations

select Track.title, Artist.name, Album.title, Genre.name  from Track join Genre join Album join Artist on Track.genre_id = Genre.id and Track.album_id = Album.id and Album.artist_id = Artist.id 

# DEMO XML AND DATABASES
- go to folder 'code' and see the trcks.zip file

In [7]:
# ASSIGNMENT: MUSICAL TRACK DATABASE
import xml.etree.ElementTree as ET
import sqlite3

conn = sqlite3.connect('track_assid2.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;
DROP TABLE IF EXISTS Genre;

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name   TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
)
''')


fname = raw_input('Enter file name: ')
if ( len(fname) < 1 ) : fname = 'Library.xml'

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
def lookup(d, content):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == content :
            found = True
    return None

stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict')
print 'Dict count:', len(all)
for entry in all:
    if ( lookup(entry, 'Track ID') is None ) : continue

    name = lookup(entry, 'Name')
    artist = lookup(entry, 'Artist')
    album = lookup(entry, 'Album')
    genre = lookup(entry,'Genre')
    count = lookup(entry, 'Play Count')
    rating = lookup(entry, 'Rating')
    length = lookup(entry, 'Total Time')

    if name is None or artist is None or album is None : 
        continue

    print name, artist, album, count, rating, length

    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Genre (name) 
        VALUES ( ? )''', ( genre, ) )
    cur.execute('SELECT id FROM Genre WHERE name = ? ', (genre, ))
    genre_id = cur.fetchone()[0]
    
    cur.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, genre_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ?, ? )''', 
        ( name, album_id, genre_id, length, rating, count ) )

    conn.commit()

    

Enter file name: 
Dict count: 404
Another One Bites The Dust Queen Greatest Hits 55 100 217103
Asche Zu Asche Rammstein Herzeleid 79 100 231810
Beauty School Dropout Various Grease 48 100 239960
Black Dog Led Zeppelin IV 109 100 296620
Bring The Boys Back Home Pink Floyd The Wall [Disc 2] 33 100 87118
Circles Bryan Lee Blues Is 54 60 355369
Comfortably Numb Pink Floyd The Wall [Disc 2] 36 100 384130
Crazy Little Thing Called Love Queen Greatest Hits 38 100 163631
Electric Funeral Black Sabbath Paranoid 44 100 293015
Fat Bottomed Girls Queen Greatest Hits 38 100 257515
For Those About To Rock (We Salute You) AC/DC Who Made Who 84 100 353750
Four Sticks Led Zeppelin IV 84 100 284421
Furious Angels Rob Dougan The Matrix Reloaded 54 100 330004
Gelle Bryan Lee Blues Is 45 60 199836
Going To California Led Zeppelin IV 100 100 215666
Grease Various Grease 42 100 205792
Hand of Doom Black Sabbath Paranoid 36 100 429609
Hells Bells AC/DC Who Made Who 82 100 312946
Hey You Pink Floyd The Wall [D

TypeError: 'NoneType' object has no attribute '__getitem__'